# テキストの埋め込みを使った類似性検索デモノートブック
埋め込みモデル：Cohere Embed Multilingual v 3.0(OCI Generative AI Searvice)

In [1]:
import oci
import os
from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd

In [2]:
_= load_dotenv(find_dotenv())

## OCI認証パラメータの設定

In [3]:
CONFIG_PROFILE = "DEFAULT" # 構成ファイルに合わせて変更してください。
config = oci.config.from_file(file_location='~/.oci/config', profile_name=CONFIG_PROFILE)
config["region"] = "us-chicago-1"

## OCI GenAIクライアントの生成

In [4]:
compartment_id = os.getenv("OCI_COMPARTMENT_ID") 

In [5]:
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

## 埋め込み処理共通のパラメータの設定

In [6]:
model_id = os.getenv("OCI_GENAI_EMBED_MODEL_ID")
print(f"model_id:{model_id}")

model_id:cohere.embed-multilingual-v3.0


In [7]:
embed_text_detail = oci.generative_ai_inference.models.EmbedTextDetails()
embed_text_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=model_id)
embed_text_detail.compartment_id = compartment_id

## 検索対象ドキュメントの埋め込み処理

### 検索対象のドキュメントの定義

In [8]:
documents = [
    "COVID-19の症状は？",
    "COVID-19には多くの症状があります。",
    "COVID-19の症状は悪いです。",
    "COVID-19の症状は良くありません",
    "COVID-19の症状は悪いです。5Gの機能には、より広範なサービスカバレッジ、より多くの利用可能な接続数、および低電力消費が含まれます。",
    "COVID-19はウイルスによって引き起こされる病気です。最も一般的な症状は発熱、悪寒、および喉の痛みですが、他にもさまざまな症状があります。",
    "COVID-19の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "認知症には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COVID-19には以下の症状があります。記憶喪失、判断力の低下、および混乱を経験すること。",
    "COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。",
    "新型コロナ感染症の症状には、発熱や悪寒、新しい持続的な咳、嗅覚や味覚の喪失や変化など、さまざまな症状が含まれます。",
    "鳥インフルエンザの症状には、高熱、咳、筋肉の痛み、倦怠感、頭痛、その他の症状が含まれます。",
    "COVID-19は、SARS-CoV-2（新型コロナウイルス）によって引き起こされる感染症です。2019年12月に最初に確認され、その後、世界的なパンデミックとなりました。症状としては、発熱、咳、息切れ、倦怠感、嗅覚や味覚の喪失などがあります。また、感染経路は、主に飛沫感染と接触感染ですが、空気感染の可能性もあります。高齢者や基礎疾患のある方は重症化リスクが高く、死亡率も高いとされています。COVID-19に対しては、ワクチン接種や手洗い、マスク着用、ソーシャルディスタンスなどの予防措置が推奨されています。パンデミックは世界中の医療システムや経済に大きな影響を与えました。",
    "SGDs（持続可能な開発目標）は、2015年に国連加盟国によって採択された「持続可能な開発のための2030アジェンダ」の中核を成す17の国際目標です。",
    "ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。",
    "著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。",
    "ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。",
    "近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。",
    "Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです"
]

In [9]:
embed_text_detail.inputs = documents

### ドキュメントの埋め込み実行

In [10]:
embed_text_detail.input_type = "SEARCH_DOCUMENT" # 検索対象のドキュメント
doc_embedding = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
doc_embedding_vectors = np.array([embed for embed in doc_embedding.data.embeddings])

In [11]:
print(f"埋め込みの数:{len(doc_embedding.data.embeddings)}")
print(f"埋め込みの次元数):{len(doc_embedding.data.embeddings[0])}")

埋め込みの数:19
埋め込みの次元数):1024


In [12]:
print(f"1件目の埋め込み:{doc_embedding.data.embeddings[0]}")

1件目の埋め込み:[-0.016326904, 0.0095825195, 0.060516357, -0.030517578, 0.0035381317, 0.028198242, -0.0041885376, -0.04107666, -0.021652222, 0.0064697266, 0.05368042, -0.014862061, 0.032989502, -0.014045715, 0.018341064, -0.000269413, 0.006450653, -0.010955811, 0.055358887, 0.04940796, 0.007423401, 0.01411438, -0.0045166016, -0.076416016, -0.013710022, 0.078308105, 0.030807495, -0.059753418, 0.028808594, -0.03894043, -0.027908325, -0.021087646, 0.049682617, -0.006828308, 0.002521515, -0.02027893, 0.0050354004, -0.04510498, 0.0077667236, 0.044708252, 0.0073890686, -0.017196655, -0.051574707, 0.036865234, -0.017166138, 0.013671875, 0.024261475, 0.050720215, 0.052581787, 0.027877808, -0.03213501, -0.009544373, 0.015823364, -0.03161621, 0.031799316, -0.016586304, -0.0054893494, 0.033325195, 0.022262573, 0.035125732, 0.015220642, -0.001168251, -0.04650879, 0.013809204, -0.01235199, 0.017684937, 0.012794495, 0.014442444, -0.010314941, 0.0030574799, 0.025314331, 0.021957397, 0.036224365, -0.0181427,

## 類似性検索のデモ（1）

### クエリーの埋め込み処理

#### クエリーの定義

In [13]:
query1 = ["ベクトルデータベースとは？"]

#### クエリー1の埋め込み生成

In [14]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector1 = np.array(query_embedding1.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [15]:
# 類似度スコアの計算
scores = np.dot(query_vector1, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [16]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8247,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
1,2,0.7664,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
2,3,0.6599,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
3,4,0.5685,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです
4,5,0.4050,COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。


## 類似性検索のデモ（2）

### クエリーの埋め込み処理

#### クエリーの定義

In [17]:
query2 = ["近似最近傍検索とは？"]

#### クエリー2の埋め込み生成

In [18]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query2
query_embedding2 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector2 = np.array(query_embedding2.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [19]:
# 類似度スコアの計算
scores = np.dot(query_vector2, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [20]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8964,近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。
1,2,0.6872,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
2,3,0.4275,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
3,4,0.3810,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
4,5,0.3495,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです


## 類似性検索のデモ（3） - おまけ（クエリーの埋め込みの平均ベクトルで検索してみる）

### クエリーの埋め込み処理

#### クエリーの定義

In [22]:
query1 = ["ベクトルデータベースとは？"]
query2 = ["近似最近傍検索とは？"]

#### クエリー1、2の埋め込み実行

In [23]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
embed_text_detail.inputs = query2
query_embedding2 = generative_ai_inference_client.embed_text(embed_text_detail)

#### クエリー1、2の埋め込みの平均を計算

In [24]:
query_vector1 = np.array(query_embedding1.data.embeddings[0])
query_vector2 = np.array(query_embedding2.data.embeddings[0])
avg_query_vector = (query_vector1 + query_vector2) / 2
print(f"埋め込みの平均のノルム（ベクトルの長さ）: {np.linalg.norm(avg_query_vector)}")
query_embedding_vector = avg_query_vector / np.linalg.norm(avg_query_vector)
print(f"正規化した埋め込みのノルムベクトルの長さ）: {np.linalg.norm(query_embedding_vector)}")

埋め込みの平均のノルム（ベクトルの長さ）: 0.836933928464785
正規化した埋め込みのノルムベクトルの長さ）: 1.0


### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [25]:
# レスポンスオブジェクトから埋め込みベクトルを抽出
doc_embedding_vectors = np.array([embed for embed in doc_embedding.data.embeddings])
# 類似度スコアの計算
scores = np.dot(query_embedding_vector, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [26]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.8048,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
1,2,0.7726,近似最近傍検索は、高次元空間内で与えられたクエリポイントに最も近い（最も類似した）データポイントを効率的に見つけるアルゴリズムです。
2,3,0.7481,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
3,4,0.6855,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
4,5,0.5484,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです


## 類似性検索のデモ（4） - クロスリンガル（言語跨りの）検索

### クエリーの埋め込み処理

#### クエリーの定義

In [ ]:
query1 = ["What is a vector database?"]
#query1 = ["¿Qué es una base de datos vectorial?"] #スペイン語

#### クエリー1の埋め込み生成

In [33]:
embed_text_detail.input_type = "SEARCH_QUERY" # 検索クエリー
embed_text_detail.inputs = query1
query_embedding1 = generative_ai_inference_client.embed_text(embed_text_detail)
# レスポンスオブジェクトから埋め込みベクトルを抽出
query_vector1 = np.array(query_embedding1.data.embeddings[0])

### 類似性検索の実行

#### 類似度スコア（コサイン類似度）の計算

In [34]:
# 類似度スコアの計算
scores = np.dot(query_vector1, np.transpose(doc_embedding_vectors))

#### 結果の表示（上位5件）

In [35]:
# スコアとドキュメントをデータフレームに変換
#top_n = len(documents)
top_n = 5
top_doc_idxs = np.argsort(-scores)[:top_n]

df = pd.DataFrame({
    'ランク': range(1, top_n + 1),
    'スコア': [f"{scores[i]:.4f}" for i in top_doc_idxs],
    'ドキュメント': [documents[i] for i in top_doc_idxs]
})
# スコアの最小値と最大値に基づいて色付けを行う
def color_scale(val):
    score = float(val)
    color = f'background-color: rgba(255, 99, 71, {score:.2f})'  # 赤色のグラデーション
    return color

# データフレームを表示（ヒートマップスタイル）
display(df.style
        .map(color_scale, subset=['スコア']) 
        .set_properties(**{'text-align': 'left'})
        .set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
            {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ]))

,ランク,スコア,ドキュメント
0,1,0.5956,ベクトルデータベースは、データをベクトル（数値の配列）として表現し、ベクトル間の類似性に基づいて検索や分析を行うことに特化したデータベースシステムです。
1,2,0.5339,著名なベクトルデータベースには、ChromaDB、Faiss、Pinecone、Annoy、Oracle Database 23ai AI Vector Search などがあります。
2,3,0.4840,Oracle Databaseとは、米国オラクル が開発・販売している、リレーショナル・データベース管理システム のことです
3,4,0.4508,ベクトルデータベースには、効率的な検索のためにHNSWのような近似最近傍検索アルゴリズムを採用しているものがあります。
4,5,0.3471,COBIT 2019は政府のデジタル・ガバメント計画を支えるガバナンス態勢を整備する際の最適な枠組みです。
